<a href="https://colab.research.google.com/github/gupta24789/chatgpt_prompts/blob/main/01_chatgpt_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predict title and summary of text

## Objective :


    Input Text :
    Output : Json Object :
    {"title": title of given news, "summary" : summary of news}

In [ ]:
import openai
import json
from pprint import pprint

## Config

In [ ]:
## Set the open ai key
openai.api_key = "<provide you api key>"

## define model
model_name = "gpt-3.5-turbo-0125"

## Method-1

In [ ]:
def generate_response(messages):
    response = {}
    response = openai.ChatCompletion.create(messages=messages, model=model_name,
                                            seed = 247,response_format={ "type": "json_object" })
    choices = response["choices"]
    completion = choices[0].message.content
    return completion

In [ ]:
def get_custom_prompt(article):

    messages = [
    {
        "role": "system",
        "content": f"""
                        You are a useful assistant. Deliver your response in valid json format which have below keys
                        title represents the title of article
                        summary represets the summary of article.
                   """
    },
    {
        "role": "user",
        "content": f"""
                    Here is an article: {article}. Please return a title and summary.
                   """
    }
    ]

    return messages


In [ ]:
article = """
Rohan Shah, Technical Analyst at Religare Broking, advised against taking risk and follow a buy-on-dips strategy.
"Any dip on  22,200 can be good buying opportunity and around 23,400 on the higher side," he said.

On March 1, the Nifty closed at an all-time high of  22,339 on the back of a positive response to the GDP numbers.

In a conversation with Moneycontrol, Upasana Chachra, Chief India Economist, Morgan Stanley, said that the trend of India doing well vis-à-vis other developed markets and emerging markets will continue as India's growth is more structural and virtuous in nature in our view.

"I think we will see more broad-based growth recovery into fiscal 25 and growth rates sustaining at these six-plus or six-and-a-half type of levels in the next few years," she added.
"""

In [ ]:
prompt = get_custom_prompt(article)
response = generate_response(prompt)

In [ ]:
pprint(json.loads(response))

{'summary': 'Rohan Shah of Religare Broking advises buying on dips around '
            '22,200 and the Nifty closes at an all-time high of 22,339 on '
            'March 1. Upasana Chachra of Morgan Stanley predicts sustained '
            "growth rates for India into fiscal 25 based on the country's "
            'structural and virtuous growth.',
 'title': 'Experts Recommend Buy-on-Dips Strategy and Positive Outlook on '
          "India's Growth"}


## Method-2

In [ ]:
def get_custom_prompt(article):

    base_prompt = """
                Please retrieve title and summary from the following news article.
                If you can't find the information from this article then return "". Do not make things up.
                For this you can use your general knowledge (it doesn't have to be from this article). Always return your
                response as a valid JSON string. The format of that string should be this,
                {
                    "title": "Stock Market",
                    "summary": "This week experts are bullish in nift"
                }
                News Article:
                ============
    """

    messages = [
    {
        "role": "system",
        "content": "You are a useful assistant"
    },
    {
        "role": "user",
        "content": base_prompt + article
    }
    ]

    return messages


In [ ]:
prompt = get_custom_prompt(article)
response = generate_response(prompt)

In [ ]:
pprint(json.loads(response))

{'summary': 'Technical Analyst advises buy-on-dips strategy for Nifty; Nifty '
            "closes at all-time high on positive GDP numbers; India's growth "
            'outlook remains strong',
 'title': 'Stock Market'}


## Method-3 : Using function calls

In [ ]:
functions = [
    {
        "name": "summary",
        "description": "Shows the title and summary of some text.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "Title of the article"
                },
                "summary": {
                    "type": "string",
                    "description": "Summary of the article"
                }
            }
        }
    }
]

In [ ]:
def get_custom_prompt(article):

    messages = [
    {
        "role": "system",
        "content": f""" You are a useful assistant"""
    },
    {
        "role": "user",
        "content": f"""
                    Here is an article: {article}. Please return a title and summary.
                   """
    }
    ]

    return messages


In [ ]:
def generate_response(messages):
    response = {}
    response = openai.ChatCompletion.create(messages=messages, model=model_name,
                                            seed = 247,
                                            functions = functions,
                                            function_call = {"name": functions[0]["name"]}
                                            )

    return response['choices'][0].message.function_call.arguments

In [ ]:
prompt = get_custom_prompt(article)
response = generate_response(prompt)

In [ ]:
pprint(json.loads(response))

{'summary': 'Rohan Shah from Religare Broking advises a buy-on-dips strategy, '
            'citing a buying opportunity around 22,200 and 23,400. Nifty '
            'closed at an all-time high of 22,339 driven by positive GDP '
            "numbers. Upasana Chachra from Morgan Stanley expects India's "
            'growth trend to surpass other markets, anticipating broad-based '
            'growth recovery into fiscal 2025.',
 'title': 'Analysis of Stock Market Trends in India'}
